## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-13-18-47-13 +0000,nypost,SpaceX to invest $2 billion in Elon Musk’s xAI...,https://nypost.com/2025/07/13/business/spacex-...
1,2025-07-13-18-34-07 +0000,nypost,Several injured in shooting at Lexington’s Blu...,https://nypost.com/2025/07/13/us-news/several-...
2,2025-07-13-18-26-53 +0000,nypost,"Megyn Kelly, MAGA stars call for Pam Bondi’s h...",https://nypost.com/2025/07/13/us-news/megyn-ke...
3,2025-07-13-18-19-57 +0000,nypost,Migrant scammers are ‘hijacking’ NYC pedicab i...,https://nypost.com/2025/07/13/us-news/migrant-...
4,2025-07-13-17-58-42 +0000,nypost,Third Avenue retail scene on upswing,https://nypost.com/2025/07/13/business/third-a...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2312/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
50,trump,8
103,against,4
7,report,4
59,new,4
139,he,3


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
35,2025-07-13-00-39-10 +0000,nypost,It’s no surprise the Trump DOJ confirmed Jeffr...,https://nypost.com/2025/07/12/business/its-no-...,28
5,2025-07-13-17-24-42 +0000,nypost,Secret Service failed to discipline agent at t...,https://nypost.com/2025/07/13/us-news/secret-s...,25
14,2025-07-13-16-27-25 +0000,nypost,Ex-Gov. David Paterson said rabid Mamdani supp...,https://nypost.com/2025/07/13/us-news/ex-gov-d...,25
34,2025-07-13-01-00-00 +0000,wsj,Economists expect stronger growth and job crea...,https://www.wsj.com/economy/economists-see-low...,24
19,2025-07-13-15-18-43 +0000,nyt,"To Sidestep Trump Tariffs, Asian Nations Seek ...",https://www.nytimes.com/2025/07/13/business/ec...,21


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
35,28,2025-07-13-00-39-10 +0000,nypost,It’s no surprise the Trump DOJ confirmed Jeffr...,https://nypost.com/2025/07/12/business/its-no-...
14,25,2025-07-13-16-27-25 +0000,nypost,Ex-Gov. David Paterson said rabid Mamdani supp...,https://nypost.com/2025/07/13/us-news/ex-gov-d...
34,24,2025-07-13-01-00-00 +0000,wsj,Economists expect stronger growth and job crea...,https://www.wsj.com/economy/economists-see-low...
5,17,2025-07-13-17-24-42 +0000,nypost,Secret Service failed to discipline agent at t...,https://nypost.com/2025/07/13/us-news/secret-s...
29,17,2025-07-13-07-53-21 +0000,nypost,New Jersey tops nation’s highest property tax ...,https://nypost.com/2025/07/13/real-estate/new-...
30,15,2025-07-13-06-27-28 +0000,nypost,"Violent brute allegedly shot parents, teen sis...",https://nypost.com/2025/07/13/us-news/new-mexi...
8,13,2025-07-13-17-08-33 +0000,nypost,Military jet’s mysterious collision raises que...,https://nypost.com/2025/07/13/us-news/military...
24,13,2025-07-13-14-20-54 +0000,nypost,Kash Patel insists Epstein ‘conspiracy theorie...,https://nypost.com/2025/07/13/us-news/kash-pat...
31,12,2025-07-13-04-42-56 +0000,nypost,DOJ drops charges against Utah surgeon accused...,https://nypost.com/2025/07/13/us-news/doj-drop...
36,12,2025-07-12-23-45-33 +0000,nyt,Air India Plane Crash Report Says Fuel to Engi...,https://www.nytimes.com/2025/07/11/world/asia/...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
